*Notebook details first attempts at scraping the web with Selenium.* 

We can use Selenium to automate actions within a web-browser. Setup was pretty easy: 

1. Install Selenium libraries with `pip install selenium`

2. Install webdriver manager with `pip install webdriver-manager`

### What is Selenium?

Selenium is a programming library that is compatible with multiple languages, including Python, C#, Ruby, and JavaScript. Often used for testing web applications, Selenium is popular amongst data scientists, developers, and software engineers alike with an interest in the creation and maintenance of applications. 

### Why use Selenium over Beautiful Soup?

The main use of Selenium is extracting various data types and elements from websites and applications in order to gain information about a topic or dataset.

Beautiful Soup is great for extracting context from websites but can run into trouble when websites are loading content after HTML using Javascript.

You can check which websites use Javascript to load content after html by installing and activating an extension like [Disable Javascript](https://developer.chrome.com/docs/devtools/javascript/disable/). As some examples: 

- Bandcamp stops the music and you can't click play
- NTS allows you to navigate links but not click any play buttons, also only loads like 12 shows at a time 
- Amazon doesn't load products, menus at all 
- AirBnb doesn't load anything!

Lets begin by installing everything we need: 

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display, clear_output

First, we need to install the latest `webdriver_manager` for the browser we would like to automate. The webdriver acts like a bridge between Selenium and the browser. I'm using Chrome in this case so that's the driver I'll be installing: 

In [3]:
service = Service(executable_path=ChromeDriverManager().install())

Starts chrome browser:

In [4]:
driver = webdriver.Chrome(service=service)


Navigate to webpage we want and wait a period of time to ensure webpage has loaded fully before we try new commands. In this case I'm searching AirBnb for a cave I can stay in anywhere for a weekend:

In [5]:
driver.get("https://www.airbnb.co.uk/?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&search_mode=flex_destinations_search&source=structured_search_input_header&search_type=category_change&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=weekend_trip&location_search=MIN_MAP_BOUNDS&category_tag=Tag%3A670")
time.sleep(1)

The following will return a BeautifulSoup object, which represents the html document as a nested data structure:

In [6]:
soup= BeautifulSoup(driver.page_source, 'html.parser')
soup

<html class="scrollbar-gutter js-focus-visible" data-hyperloop-version="1" data-is-hyperloop="true" dir="ltr" lang="en-GB" style="--vh:6.53px; --vw:12px; --vw-unitless:1200; --vw-px:1200px;"><head><meta charset="utf-8"/><meta content="en-GB" name="locale"/><meta content="notranslate" name="google"/><meta content="authenticity_token" id="csrf-param-meta-tag" name="csrf-param"/><meta content="null" id="csrf-token-meta-tag" name="csrf-token"/><meta content="" id="english-canonical-url"/><meta content="on" name="twitter:widgets:csp"/><meta content="yes" name="mobile-web-app-capable"/><meta content="yes" name="apple-mobile-web-app-capable"/><meta content="Airbnb" name="application-name"/><meta content="Airbnb" name="apple-mobile-web-app-title"/><meta content="#ffffff" name="theme-color"/><meta content="#ffffff" name="msapplication-navbutton-color"/><meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/><meta content="/?utm_source=homescreen" name="msapplication-star

Now we can extract the listings on the AirBnb page from within our Soup object easily because they have the `itemprop` attribute with the `itemListElement` value:

In [7]:
listings = soup.select('[itemprop="itemListElement"]')

In [8]:
def property_info(listings):
    df = pd.DataFrame(columns = ['Location' , 'Price', 'Guests' , 'Bedrooms', 'Beds', 'Bathrooms'])
    for i,listing in enumerate(listings):
        details = listing.findAll(text=True)
        location, price = details[1], details[4]
        url = listing.select_one('[itemprop="url"]')['content']
        driver.get("https://" + str(url))
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        amneties = []
        for element in soup.findAll('li'):
            for text in element.findAll(text=True):
                if ' · ' not in text:
                    amneties.append(text)
        row = [location, price]+amneties
        df.loc[len(df)] = row[0:6]
        time.sleep(1)
        clear_output(wait=True)
        display(str(100*(i+1)/len(listings)) + "% complete")
    return(df)

In [9]:
caves = property_info(listings)
caves

'100.0% complete'

,Location,Price,Guests,Bedrooms,Beds,Bathrooms
0,"Noyers-sur-Cher, France",£225,4 guests,1 bedroom,3 beds,1 bathroom
1,"Bourré, France",£192,3 guests,1 bedroom,1 bed,1 bathroom
2,"Brissac-Loire-Aubance, France",£769,8 guests,4 bedrooms,4 beds,3 bathrooms
3,"Jaunay-Marigny, France",£348,6 guests,2 bedrooms,4 beds,1 bathroom
4,"Noizay, France",£187,4 guests,1 bedroom,2 beds,1 bathroom
5,"Nazelles-Négron, France",£277,6 guests,2 bedrooms,4 beds,1 bathroom
6,"Noizay, France",£188,3 guests,1 bedroom,2 beds,1 bathroom
7,"Nazelles-Négron, France",£226,2 guests,1 bedroom,1 bed,1 private bathroom
8,"Vouvray, France",£499,6 guests,3 bedrooms,4 beds,2 bathrooms
9,"Montrichard, France",£220,4 guests,1 bedroom,2 beds,1 bathroom
